In [1]:
from manim import *
config.media_width = "100%"


# Introduction
* Imagine you can get $1,000,000 only by solving familiar yet not simple equations

> write the equation on the chalkboard in university.

* For example, if I told you to predict for me what trajectory the ball will take, I would give you $10  
given the initial conditions 

> film a scene where you shoot a ball in a field, a snapshot of the ball before hitting it.

* That should be simple enough. Newton's second law! 

## Trajectory Path

In [3]:
%%manim --disable_caching -v WARNING -qh Newton

#acceleration to vector field
class Newton(MovingCameraScene):
    def construct(self):

        # set up axes and graph
        axes = Axes(
            x_range=[0, 4],
            y_range=[-1, 3],
            axis_config={"color": WHITE}, 
        )

        t_label = MathTex("t").next_to(axes.x_axis, RIGHT)
        s_label = MathTex("s(t)").next_to(axes.y_axis, UP)
        labels= VGroup(t_label, s_label)
        
        f= lambda x: -3/4 *(x-2)**2 +3
        graph = axes.plot(f, color=RED)
        plane=VGroup(axes,graph)

        
        

        # set up point and vector

        def get_tangent_vector(proportion, curve, scale=1):
            coord_i = curve.point_from_proportion(proportion if proportion <0.999 else proportion - 0.001)
            coord_f = curve.point_from_proportion(proportion + 0.001 if proportion <0.999 else proportion ) 
            reference_line = Line(coord_i,coord_f)
            unit_vector = reference_line.get_unit_vector() * scale
            vector = Arrow(coord_i, coord_i + unit_vector,color= YELLOW, buff=0)
            return vector

        vector = get_tangent_vector(0, graph, scale=1.5)

        dot = Dot(graph.point_from_proportion(0), color=YELLOW).scale(3)
        dot.add_updater(lambda d: d.move_to(vector.get_start()))

        def tan(mob, alpha):
            mob.become(
                    get_tangent_vector(alpha,graph,scale=1.5))

        def udp(mob):
            mob.next_to(dot.get_center(),DOWN,buff=0.25)

        gravity=Arrow(dot.get_center(),dot.get_center()+1.5*DOWN,color=GREEN)
        gravity.add_updater(udp)
        

       
        u= MathTex(r'u(t)', color=YELLOW).next_to(dot.get_center(), UP)
        g= MathTex(r'g', color=GREEN).next_to(gravity.get_center(), RIGHT)

        u.add_updater(lambda l: l.next_to(dot.get_center(), UP,buff=0.9))
        g.add_updater(lambda l: l.next_to(gravity.get_center(), RIGHT,buff=0.5))
        self.camera.frame.save_state()
    


        self.camera.frame.move_to(dot.get_center())
        self.camera.frame.scale(0.5)

        self.play(Create(dot))
        self.play(LaggedStartMap(Create,VGroup(vector,gravity),lag_ratio=1),rate_func=smooth)
        self.play(LaggedStartMap(Write,VGroup(u,g),lag_ratio=0.5))
        self.play(Restore(self.camera.frame))
        self.bring_to_back(plane)

        self.play(Create(plane),run_time=2,rate_func=linear)
        self.play(Write(labels))
        

        
        



        self.play(UpdateFromAlphaFunc(vector, tan), run_time=8, rate_func=linear)


Manim Community v0.19.0

Now, same question, but imagine I give you a snapshot of a flow of a river (Seaton Park walking during the day). How would you do it?

![river](river.jpg)

That was the one million prize problem that puzzled mathematicians and physicists
Which is the navier-stokes equations
> back to the chalkboard with equation focus

# Scene 2
As complicated as it seems, it is all about fundamental laws of physics that we know.
> Explain Navier-Stokes through animations in Manim.


In [4]:
%%manim --disable_caching -v WARNING -qh Navier
class Navier(Scene):
    def construct(self):
        navier_stokes= Text('Naiver-stokes Equations').scale(1.5).shift(ORIGIN)
        T1= Text('Continunity equation')
        T2= Text('Momentum equation')
        self.play(Write(navier_stokes,run_time=4, rate_func=smooth))
        self.play(navier_stokes.animate.to_edge(UP))
        self.wait()

        
        ns=MathTex(r'\rho',r'( \frac{\partial u}{ \partial t} + u \cdot \nabla u)', r'=', r'-\nabla P', r'+ \mu \nabla^2\textbf{u}', r'+ \textbf{f}', color=WHITE).scale(1.5)
      
        ce_ns= MathTex(r' \nabla \cdot \textbf{u} =0',color=WHITE).scale(2)
        

        vg=VGroup(T1,ce_ns,T2,ns)
        vg.arrange(DOWN).to_edge(DOWN,buff=1)
        vg[:2].shift(0.5*UP)
        vg[2:].shift(0.3*DOWN)
        self.play(LaggedStartMap(Write,vg,run_time=4,lag_ratio=0.3))
        
        self.wait(3)
        self.play(LaggedStartMap(FadeOut,self.mobjects))


Manim Community v0.19.0

But we have to state these are the simplified version under the assumptions;

 * incompressible

 * Newtonian 

 * isothermal.

In [5]:
def div_field(p):
    epsilon=1
    x , y , z =  p
    # Source-like terms
    fx1 = (x - 2) / ((x - 2)**2 + (y - 2)**2 + epsilon)
    fy1 = (y - 2) / ((x - 2)**2 + (y - 2)**2 + epsilon)

    fx2 = (x + 2) / ((x + 2)**2 + (y + 2)**2 + epsilon)
    fy2 = (y + 2) / ((x + 2)**2 + (y + 2)**2 + epsilon)

    # Sink-like terms
    fx3 = (x - 2) / ((x - 2)**2 + (y + 2)**2 + epsilon)
    fy3 = (y + 2) / ((x - 2)**2 + (y + 2)**2 + epsilon)

    fx4 = (x + 2) / ((x + 2)**2 + (y - 2)**2 + epsilon)
    fy4 = (y - 2) / ((x + 2)**2 + (y - 2)**2 + epsilon)

    # Net vector
    fx = fx1 + fx2 - fx3 - fx4
    fy = fy1 + fy2 - fy3 - fy4

    return np.array([fx, fy,0])

In [6]:
def pointers(p, n=5, radius=0.5,sign=1):
    terminal_point = p
    vectors=VGroup()
    for i in range(n):
            angle = 2 * PI * i / n
            origin = terminal_point + radius * np.array([np.cos(angle), np.sin(angle), 0])
            new_terminal = terminal_point + 1.2*np.array([np.cos(angle), np.sin(angle), 0])
            vector = Arrow(start=origin, end=new_terminal , buff=0, color=BLUE)
            if sign == 1:
                vector.rotate(PI)
            vectors.add(vector)
    return vectors


In [15]:
def stokes_flow(v,R=0.7,U=2):
    
    x,y , z = v
    """
    Creeping flow (Re << 1) around a cylinder of radius R
    in 2D at uniform flow speed U.
    """
    r = np.sqrt(x**2 + y**2)
    theta = np.arctan2(y, x)
    
    if r < R:
        return np.array([0.0, 0.0,0.0])  # inside the cylinder, no flow

    # Radial and angular velocity components (u_r, u_theta)
    ur = U * (1 - (R**2 / r**2)) * np.cos(theta)
    ut = -U * (1 + (R**2 / r**2)) * np.sin(theta)

    # Convert to Cartesian
    u = ur * np.cos(theta) - ut * np.sin(theta)
    v = ur * np.sin(theta) + ut * np.cos(theta)

    return np.array([u, v, 0.0])


In [11]:
%%manim --disable_caching -v WARNING -qh Assumptions
class Assumptions(Scene):
    def construct(self):
        title = Text(
            "Assumptions for this video", color=BLUE)
        title.scale(1.25)
        title.shift(3 * UP)

        l = BulletedList("Newtonian", "Incompressible",
                        'Isothermal', height=3,width=3)
        l.scale(1.5)
        l.shift(0.25*DOWN)

        self.play(FadeIn(title,shift=3*UP))
        self.play(Write(l))
        self.wait()

        self.play(l.animate.fade_all_but(0))
        self.wait()

        self.play(l.animate.fade_all_but(1))
        self.wait()

        self.play(l.animate.fade_all_but(2))
        self.wait()

Manim Community v0.19.0

In [12]:
%%manim --disable_caching -v WARNING -qh fluid
class fluid(Scene):
    def construct(self):
        ax=Axes(x_range=[0,6],y_range=[0,5], axis_config={
                "include_tip": False}).shift(0.5*UP)
        f= lambda x : 3 
        xlbl= ax.get_x_axis_label(MathTex(r'\tau').scale(2),edge=DOWN, direction=DOWN, buff=0.5)
        ylbl= ax.get_y_axis_label(MathTex(r'\mu').scale(2),edge=LEFT, direction=LEFT, buff=1).rotate(PI/2)

        graph= ax.plot(f,color=BLUE)

    

        nw=Text('Newtonian',color=BLUE).next_to(graph,UP)
       
        vg=VGroup(ax,xlbl,ylbl,graph,nw)
        self.play(LaggedStartMap(Write,vg,lag_ratio=0.5),run_time=4,rate_func=smooth)
       
        self.wait()

Manim Community v0.19.0

the first about mass conservation

In [13]:
%%manim --disable_caching -v WARNING -ql Compressible
class Compressible(Scene):
    def construct(self):   
        plane=NumberPlane()
        divP=MathTex(r'\nabla \cdot \textbf{u} < 0', color=WHITE)
        divN=MathTex(r'\nabla \cdot \textbf{u} > 0', color=WHITE)

        points= VGroup()
        for i in np.arange(-7,7.5,0.5):
            for j in np.arange(-4, 4.5,0.5):
                point = Dot(plane.c2p(i,j), color=BLUE, radius=0.03)
                points.add(point)

        points.fade(0.5)
        self.play(LaggedStartMap(FadeIn, points, run_time=2, lag_ratio=0.1))
        

        
        st=StreamLines(div_field, stroke_width=2,x_range=[-10, 10, 0.5], y_range=[-6, 6, 0.5])
        vf= ArrowVectorField(div_field, x_range=[-10, 10, 0.5], y_range=[-6, 6, 0.5])

        self.play(Write(vf, run_time=3,rate_func=smooth))
        self.play(FadeOut(points))
        


        v1= pointers([-2.3,2.2,0], n=10, radius=0.5)
        v2= pointers([-2.3,-2.2,0], n=10, radius=0.5, sign=-1)
        v3= pointers([2.3,2.2,0], n=10, radius=0.5, sign=-1)
        v4= pointers([2.3,-2.2,0], n=10, radius=0.5)
        vg=VGroup(v1,v2,v3,v4)

        self.add(st)
        st.start_animation(warm_up=True, flow_speed=1)
        self.wait(5)
        self.play(LaggedStartMap(Create, vg, run_time=2, lag_ratio=0.1))
        self.wait(2)
        divP.next_to(v1, DOWN)
        divN.next_to(v2, UP)
        self.play(Write(divP), run_time=1)
        self.play(Write(divN), run_time=1)
        self.wait(2)
        self.play(LaggedStartMap(FadeOut, vg, run_time=2, lag_ratio=0.1))

Manim Community v0.19.0

In [16]:
%%manim --disable_caching -v WARNING -qh incompressible
class incompressible(Scene):
    def construct(self):   
        st=StreamLines(stokes_flow, stroke_width=2,x_range=[-10, 10, 0.5], y_range=[-6, 6, 0.5])
        vf= ArrowVectorField(stokes_flow, x_range=[-10, 10, 0.5], y_range=[-6, 6, 0.5])
        self.add(vf, st)
        div=MathTex(r'\nabla \cdot \textbf{u} = 0', color=WHITE).scale(2)
        self.add(div)
        
        st.start_animation(warm_up=False, flow_speed=.75)
        self.wait(5)



Manim Community v0.19.0

the second about the momentum 

Need to write someting about what each means in brief

In [17]:
%%manim --disable_caching -v WARNING -qh momentum

class momentum(Scene):
    def construct(self):
        title=Text('Navier-Stokes Momentum Equation',color=BLUE).to_edge(UP,buff=1)
        ns=MathTex(r'\rho', r'( \frac{\partial u}{ \partial t} + u \cdot \nabla u)'
        , r'=', r'-\nabla P', r'+ \mu \nabla^2\textbf{u}', r'+ \textbf{f}', color=WHITE).scale(1.5)
        nw=MathTex( r'm \textbf{a}',r'=\Sigma \textbf{F}',color=WHITE).scale_to_fit_width(ns.width)
        nw.fade(0.5)
        self.play(Write(title))
        self.play(Write(ns))
        self.wait()
        rec1= SurroundingRectangle(ns[:2])
        rec2= SurroundingRectangle(ns[2:])
        self.play(Write(rec1),rate_func=there_and_back,run_time=2)
        self.play(FadeIn(nw[0]))
     
        self.play(Write(rec2),rate_func=there_and_back,run_time=2)
        self.play(FadeIn(nw[1]))
        self.play(LaggedStart(FadeOut(ns),nw.animate.set_opacity(1),lag_ratio=0.3))
        self.wait()

Manim Community v0.19.0

## LHS
let ‘s break the intuition behind each term
First in continuum mechanics we consider density instead of mass 


In [18]:
%%manim --disable_caching -v WARNING -qh ball
# newton to navier stokes reasoning
class ball(MovingCameraScene):
    def construct(self):

        #object
        obj = Circle(radius=0.3, color=RED,fill_opacity=1).scale(2)
        surface = Line(start=2*LEFT + obj.get_bottom(), end=2*RIGHT + obj.get_bottom(), color=BLUE)

        #forces
        weight = Arrow(start=2*DOWN , end=obj.get_center(), max_tip_length_to_length_ratio=0.2 , buff=0).rotate(PI)
        weight.scale(0.8,about_point=weight.get_start())
        force = Arrow(start=2*RIGHT , end=obj.get_center(), max_tip_length_to_length_ratio=0.2 , buff=0).rotate(PI)
        force.scale(0.8,about_point=force.get_start())
        friction= Arrow(start=surface.get_start() -0.8*LEFT, end=obj.get_bottom(), max_tip_length_to_length_ratio=0.2 , buff=0)

        #text
        sym= MathTex(r'\textbf{F}',r'W', r'f', color=WHITE)
        sym[0].next_to(force, RIGHT)
        sym[1].next_to(weight, DOWN)
        sym[2].next_to(friction, DOWN)
        vg=VGroup(obj, surface, force, weight, friction)
        self.play(Create(vg), run_time=2, rate_func=linear)
        self.play(Write(sym),run_time=2)

        nw=MathTex( r'\Sigma \textbf{F}',r' = m \textbf{a}',color=WHITE).scale(2).to_corner(UL)
        self.play(Write(nw))
        
        self.wait(2)


Manim Community v0.19.0

In [19]:
class AbstractExample(ZoomedScene):
    def __init__(self, **kwargs):
        ZoomedScene.__init__(
            self,
            zoom_factor=0.3, # scale of zoomed_camera camera
            zoomed_display_height=2,
            zoomed_display_width=3,
            zoomed_camera_config={
                "default_frame_stroke_width": 2, # frame border
            },
            **kwargs
        )

    def setup(self):
        # Call ZoomedScene.setup(self)
        super().setup()
        # Define zoom mobs
        zoomed_display        = self.zoomed_display
        zoomed_display_border = zoomed_display.display_frame
        frame                 = self.zoomed_camera.frame
        # set zoom mobs props
        
        self.zoomed_objs = [zoomed_display, zoomed_display_border, frame]

In [20]:
%%manim --disable_caching -v WARNING -qh density
# newton to navier stokes reasoning
class density(AbstractExample):
    def construct(self):

        #part of the camera
        zoomed_display, zoomed_display_border, frame = self.zoomed_objs
        zoomed_display.scale(0.85)


        #the water
        fluid=VGroup()
        
        x_min, x_max = -2, 2
        y_min, y_max = -2, 1

        for j in np.linspace(y_min, y_max, 40):
            for i in np.linspace(x_min, x_max, 40):
                point = Dot([i, j, 0],radius=0.04, color=BLUE)
                fluid.add(point)

        bottom_left = [x_min-0.1, y_min-0.1, 0]
        bottom_right = [x_max+0.1, y_min-0.1, 0]
        top_right = [x_max+0.1, y_max+0.1, 0]
        top_left = [x_min-0.1, y_max+0.1, 0]

        # Create lines (skip the top line)
        left_edge = Line(bottom_left, top_left, color=WHITE)
        bottom_edge = Line(bottom_left, bottom_right, color=WHITE)
        right_edge = Line(bottom_right, top_right, color=WHITE)
        box = VGroup(left_edge, bottom_edge, right_edge)

        vg=VGroup(fluid, box).shift(DOWN)
        self.add(vg)
        vgc=vg.copy().scale(0.5)
        
        line= Line(start=LEFT, end=RIGHT, color=WHITE).scale(1.2)
        line.next_to(zoomed_display.get_bottom(),DOWN)
        frame.shift(vg.get_center())
        

        self.play(Create(frame))
        self.activate_zooming()
        self.play(self.get_zoomed_display_pop_out_animation())
        self.play(Create(line))
        self.play(vgc.animate.next_to(line.get_center(),DOWN), run_time=2)

        rho= MathTex(r'\rho =', color=WHITE).scale(2)
        rho.next_to(line, LEFT,buff= 1)
        self.play(Write(rho))
        density= MathTex(r'm',r'V', color=WHITE).scale(2)
        density[0].next_to(zoomed_display, UP,buff=0.1)
        density[1].move_to(vgc.get_center())
        
        self.play(LaggedStart(Write(density[0]),Write(density[1]), run_time=2, lag_ratio=0.5))
        self.wait(3)


Manim Community v0.19.0

Therefore same logic should be applied for velocity
So in this case we need to consider the whole continuum as single points with magnitudes and directions 
> acceleration scene


In [21]:
%%manim --disable_caching -v WARNING -ql velocityfield
class velocityfield(Scene):
    def construct(self):
        
        plane = NumberPlane().fade(1)
        self.add(plane)


        points= VGroup()
        for i in np.arange(-7,7.5,0.5):
            for j in np.arange(-4, 4.5,0.5):
                point = Dot(plane.c2p(i,j), color=BLUE, radius=0.05)
                points.add(point)

        points.fade(0.5)


        func = lambda p: np.array([p[1],p[0],0])
        gradf= lambda p: np.array([1.,1.,0])

        gradient_field = ArrowVectorField(gradf, color=BLUE)
        vector_field= ArrowVectorField(func)
        gradient_field.fade(1)
        vector_field.fade(1)
        self.add(gradient_field,vector_field)
        

        self.play(LaggedStartMap(FadeIn, points, run_time=2, lag_ratio=0.1))
        self.play(plane.animate.set_opacity(0.3))
        self.play(vector_field.animate.set_opacity(1))

        u= MathTex(r'u \cdot \nabla u', color=WHITE).scale(2).to_edge(UP)
        self.play(Write(u))
        self.add_foreground_mobject(u)

        self.play(LaggedStart(gradient_field.animate.set_opacity(1),vector_field.animate.fade(0.7), run_time=3, lag_ratio=0))
        self.wait(2)
        self.play(vector_field.animate.set_opacity(1))
        self.wait(2)

        vg=VGroup(vector_field, gradient_field)
        self.play(vg.animate.set_opacity(0.7))
        
 
        # streamlines
        stream_lines = StreamLines(func, stroke_width=2).fade(0.3)
        self.add(stream_lines)
        
       
        stream_lines.start_animation(warm_up=True, flow_speed=0.75)
        t=Variable(0,MathTex('t'), num_decimal_places=1).to_corner(UL)
        t.add_background_rectangle(color=BLACK)
        self.play(Write(t))
        self.play(t.tracker.animate.set_value(10),run_time=10,rate_func=linear)
        self.wait(0.5)



Manim Community v0.19.0

## RHS

We so are done from the LHS
How about the other side?
> chalkboard scene
>>The first two terms are the internal forces 
>> Which are The pressure gradient—we all know how water moves in a tube.


It moves from high pressure to low pressure, which is represented mathematically as a derivative.


In [22]:
%%manim --disable_caching -v WARNING -ql pressure

class pressure(Scene):
    def construct(self):
        points= VGroup()
        x_min, x_max = -5, 5.5
        y_min, y_max = -2, 2.5


        for i in np.arange(x_min,x_max,0.5):
            for j in np.arange(y_min, y_max,0.5):

                alpha =(i - x_min) / (x_max - x_min)
                color = interpolate_color(BLUE, RED, alpha)
                point = Dot([i,j,0], color=color, radius=0.1)
                points.add(point)

        self.add(points)
        pressure= MathTex(r'P', color=WHITE).scale(2)
        pressure.to_edge(UP)
        self.add(pressure)

        arrow= Arrow(start=2*RIGHT, end=2*LEFT, color=WHITE).scale(1.2)
        self.play(GrowFromEdge(arrow, RIGHT),rate_func= linear, run_time=5)
        self.wait()

Manim Community v0.19.0

In [23]:
%%manim --disable_caching -v WARNING -ql gradient
# Gradient vector field

class gradient(ThreeDScene):
    def construct(self):
        self.set_camera_orientation(phi=70*DEGREES, theta=-120*DEGREES)

        # Axes
        axes = ThreeDAxes(x_range=(-2, 2, 1), y_range=(-2, 2, 1), z_range=(-2, 2, 1))

        # Surface function
        func = lambda u, v: axes.c2p(u, v, 2 * np.sin(u) * np.cos(v))

        surface = Surface(
            func,
            resolution=30,
            v_range=[-1.5, 1.5],
            u_range=[-1.5, 1.5],
        ).fade(0.5)

        # Gradient in (u, v) space
        def gradient(u, v):
            dx = 2 * np.cos(u) * np.cos(v)
            dy = -2 * np.sin(u) * np.sin(v)
            return np.array([dx, dy])

        # Initial position in (u, v)
        u_val = ValueTracker(0.5)
        v_val = ValueTracker(0)
        
        pressure = MathTex(r' - \nabla P ')
        pressure.scale(2).to_corner(UL)
        self.add_fixed_in_frame_mobjects(pressure)
        # Dot on the surface
        dot = always_redraw(lambda: Dot3D(func(u_val.get_value(), v_val.get_value()), color=RED))

        
        arrow= always_redraw( lambda: Arrow(start=dot.get_center(), end=dot.get_center()-[1,1,0], color=RED))

        # Update function to simulate gradient descent in (u, v) space
        def update_uv(mob, dt):
            u = u_val.get_value()
            v = v_val.get_value()
            grad = gradient(u, v)
            step_size = 0.1 * dt  # adjust for smoothness
            u_val.set_value(u - step_size * grad[0])
            v_val.set_value(v - step_size * grad[1])


        dot.add_updater(update_uv)

        surface.set_fill_by_value(axes, colorscale= [(BLUE,0), (RED,0)],axis=2)


        self.add(axes, surface, dot, arrow)

        self.wait(10)
        dot.clear_updaters()

        self.wait()

Manim Community v0.19.0

As for the other term 
Which is the diffusion term
There is always a friction, which is represented as viscosity in fluid
> honey scene and water pouring

The tendency of water to move along it layers

> viscosity


In [24]:
%%manim --disable_caching -v WARNING -ql viscosity

class viscosity(Scene):
    def construct(self):
        #sign object
        fluid=VGroup()
        
        x_min, x_max = -6*4, 6*4
        y_min, y_max = -3, 3

        for j in range(y_min, y_max, 1):
            for i in range(x_min, x_max, 1):
                point = Dot([i, j, 0],radius=0.5, color=BLUE)
                fluid.add(point)

        fluid.shift(LEFT*4)
        fluid.fade(0.5)
        

        gs=VGroup(*[ fluid[i:i+12*4] for i in range(0, len(fluid), 12*4)])
        


    
        # self.add(fluid)
        self.add(gs)
        
        rec= SurroundingRectangle(fluid, color=WHITE, buff=0.1)
     
        
        self.add(rec)
        
        def shift(mob, alpha):
            mob.shift(RIGHT*alpha/10)

        anim=[]
        for i in range(0, len(gs)):
            animation= UpdateFromAlphaFunc(gs[i], shift, run_time=10, rate_func=linear)
            anim.append(animation)

        anim.reverse()
        viscosity= MathTex(r'\tau = \mu \frac{du}{dy}', color=WHITE).scale(4)
        self.add_foreground_mobject(viscosity)

        self.play(LaggedStart(*anim, lag_ratio=0.05))
        self.wait(2)
    
    

Manim Community v0.19.0

I love this term because it perfectly match its meaning

‘’The rate on how water diffuse relative to its layers’’

It describes how velocity spreads out between neighboring layers, capturing internal friction. How Laplacian represents diffusion 
> show a 3d animation of a function that being stretch

Because you know what is 1st derivative in 2D, so you can understand 2nd (which measures the convavity) as the average rate of change between two neighboring points.


In [25]:
%%manim --disable_caching -v WARNING -ql diffusion

class diffusion(Scene):
    def construct(self):
        ax=Axes(x_range=[0,2*PI],y_range=[-1,4])
        t=ValueTracker(1)
        f= lambda x :  t.get_value()* (-1.9*np.sin(x-1) -0.8*np.sin(3.1*x)-1*np.sin(-0.8*x))+1
        f_f= lambda x : 1
        final=ax.plot(f_f)
      
        
        graph= always_redraw(lambda : ax.plot(f,color=BLUE))
        val= [0.65,1.35,2.59,3.68,4.44,5.6]
        points=always_redraw( lambda : VGroup(*[Dot(ax.c2p(x,f(x))) for x in val]))
        p_f=VGroup(*[Dot(ax.c2p(x,f_f(x))) for x in val])

        arrows=always_redraw( lambda : VGroup(*[Arrow(points[i].get_bottom(),p_f[i],color=YELLOW) for i in range(0,len(points))]))
        
            
          
        vg=VGroup(ax,graph)
        self.play(LaggedStartMap(Create,vg))
        self.play(FadeIn(points,lag_ratio=1))
        self.play(FadeIn(arrows))
       
       

        self.play(t.animate.set_value(0),run_time=5,rate_func=linear)
        self.wait()      


Manim Community v0.19.0

In [26]:
%%manim --disable_caching -v WARNING -ql laplacian

class laplacian(ThreeDScene):
    def construct(self):
       
        D3=ThreeDAxes()
        self.set_camera_orientation(phi=45*DEGREES, theta=45*DEGREES)
        surface=Surface(lambda u,v: np.array([u,v,np.sin(u)+np.cos(v)]),u_range=[-3*PI,3*PI],v_range=[-3*PI,3*PI],resolution=30).fade(0.5)
        def get_surface(mob,alpha):
            t=1-alpha
            sin=Surface(lambda u,v: np.array([u,v,t*np.sin(u)+t*np.cos(v)]),u_range=[-3*PI,3*PI],v_range=[-3*PI,3*PI],resolution=30).fade(0.5)
            mob.become(sin)

        
        self.play(Create(surface))
        
        self.play(UpdateFromAlphaFunc(surface,get_surface,run_time=10,rate_func=linear))
        self.wait()
        
      

Manim Community v0.19.0

* And finally external forces, which could be anything, really

> $ f = \rho g$
or maxwell equations


![maxell](images.png)

>which will yield electromagnetohydrodynamics which helped us in predicting stars 

Despite the horrifying math terms, all the equation is saying for the first one is the mass is conserved and the second is mass X acceleration is equal to the internal forces on the fluid and external forces which is really just newton's second law but for fluid 



# Scene 3
Ok, then, if it is all fundamental laws of physics, why does it have a $1,000.00 prize?
The problem with this equation is this bastard (convective term).
This term makes the equation very sensitive because it encapsulates nonlinearity.
Nonlinearity tells us a lot about the turbulent nature of fluid, which we can't predict due to the chaotic behavior of the fluid. 

> Scene about the cigarette smoking.


Therefore we can tell if the fluid will behave nicely in the long-term or will give us smooth solution with any slight change in initial conditions
It makes the equation so sensitive to change (similar to the factorial function).
> show a scene when you drink coffee and swirl the cream.

> [Apply smooth stirring to coffee.]
If you consider the motion of coffee in this cup with slow stirring,
It is predictably non-turbulent; it moves smoothly, maintaining structure.
However, if we change the condition so slightly, will the math hold?

> [Apply fast stirring]
The cream begins to stretch, fold, and twist unpredictably. It is becoming turbulent.
Eventually they form a tangled mess of swirls.

These equations work beautifully in calm water or with slow velocities, but not for others
> (throw ink in water)

ink in water dispersing smoothly… then suddenly breaking up into chaotic swirls

Plus, these nonlinear PDE is insanely hard to solve.


# Ending
Therefore the problem state is “a proof of the existence of a smooth solution and uniqueness.”

Any advancement in understanding the N-S equation will yield you a fortune.

Because Navier-stokes will help us unveil what is mean by turbulence, it is not just a physics problem, but rather a math one
"Maybe the answer lies in a simple idea no one has seen yet.

"One equation. One solution. One million dollars."



In [27]:
%%manim --disable_caching -v WARNING -ql closeup
class closeup(Scene):
    def construct(self):
        title = Text('Millennium Prize Problems').to_edge(UP)
        conditions=BulletedList(r'Existence of a vector velocity field\\and a scalar pressure field',
         r'Both fields are smooth\\and globally defined', 
         'Uniqueness of the solutions'
         ,height=5,width=7
         )
        conditions.to_edge(LEFT,buff=2).shift(DOWN*0.5)
      
    
        img=ImageMobject('millenium problem.png').shift(LEFT+ 0.5 * DOWN)
        img.move_to(conditions.get_center())
        self.add(img)
        self.play(Write(title),run_time=2)
        self.play(LaggedStart(img.animate.to_edge(RIGHT),Write(conditions), lag_ratio=0.2, run_time= 5 , rate_func=smooth ))
        self.wait()

Manim Community v0.19.0

Finally despite our lack of understanding of N-S equations it still one of the breakthroughs for humankind that depict almost everything from air flows to liquid flows

And has vast amount of practical applications in real world, even though it is just approximations

We can be more greedy.

Since it described the motion of every fluid in the universe 
You can have one universal equation that can predict the behavior of everything consistently

A nice way to put it is, “Solving Navier-Stokes equations is like solving a personality.”
> sitting with Dilnaz discussing about things [ the idea is you cant predict what people will say]
